In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0 = todos los mensajes, 1 = INFO, 2 = WARNING, 3 = ERROR
import numpy as np
import keras
from IPython.display import Image, display
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from matplotlib import cm
from scipy.ndimage import label

In [2]:
from tensorflow.keras.models import Model
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from data import part_classes, animals

In [4]:
IMG_SIZE = (360,360)#)468)
total_pixels = IMG_SIZE[0]*IMG_SIZE[1]
pixel_region = round(total_pixels*0.02)

In [5]:
OPTION = 1
if OPTION == 1:
    model_name = 'xception_without_model'
    data_metrics_file = 'dataset_info_parts_metrics_methods_without'
else:
    model_name = 'xception_model'
    data_metrics_file = 'dataset_info_parts_metrics_methods'

In [6]:
model = tf.keras.models.load_model(model_name+'.h5')
conv_layers= [l.name for l in model.layers if l.__class__.__name__.endswith('Conv2D')]
dense_layers= [l.name for l in model.layers if l.__class__.__name__.endswith('Dense')]
last_conv_layer_name = conv_layers[-1]

In [7]:
df = pd.read_csv('dataset_info_parts.csv')
PATH_data = 'dataset/'
IMG_RESIZE_PATH_data = PATH_data+'images_resize/'
ANN_RESIZE_PATH_data = PATH_data+'annotations_resize/'
IMG_SIZE = (360,360)#468)
BATCH_SIZE = 32
df['file_path'] = IMG_RESIZE_PATH_data + df['file']+'.jpg'
animals = ['sheep', 'dog', 'cow', 'cat', 'horse']

df[['pred_id', 'pred_ok', 'cat_id']] = np.nan
df[['mean_act_obj', 'mean_act_back', 'perc_act_obj', 'perc_act_back']] = np.nan#

In [8]:
array = np.array([])
array_y = np.array([])
for animal in animals:
    df_a = df[df['cat']==animal]
    cat_id = animals.index(animal)
    count = 0
    for index, row in df_a.iterrows():
        img = cv2.cvtColor(cv2.imread(row.file_path),cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, IMG_SIZE)
        if len(array)==0:
            array=np.expand_dims(img.astype(np.float32), axis=0)
        else:
            array = np.concatenate([array, [img]],axis=0)
        if len(array_y)==0:
            array_y = np.array([cat_id])
        else:
            array_y = np.concatenate([array_y, [cat_id]],axis=0)
        count += 1
        if count>3:
            break     
            

In [9]:
data = {}
all_stats = []
for conv_layer in conv_layers+dense_layers:
    intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(conv_layer).output)
    activations = intermediate_layer_model.predict(array,batch_size=2)  # (1000, h, w, c)
    if 'conv' in conv_layer:
        n_samples, h, w, c = activations.shape
        flattened = activations.reshape(n_samples, -1)  # (1000, h*w*c)
        mean_per_filter = flattened.reshape(n_samples, h*w, c).mean(axis=1)
    else:
        n_samples, d = activations.shape
        mean_per_filter = activations.reshape(n_samples, -1)  # (1000, h*w*c)
        #mean_per_filter = flattened.reshape(n_samples, d).mean(axis=1)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(mean_per_filter)
    mi_scores = mutual_info_classif(X_scaled, array_y)
    data[conv_layer] = mi_scores
    """
    plt.figure(figsize=(12, 5))
    plt.bar(range(len(mi_scores)), mi_scores)
    plt.xlabel("Índice de filtro (canal)")
    plt.ylabel("MIS con clase")
    plt.title(f"Mutual Information Score - Capa: {conv_layer}")
    plt.show()
    """
    summary_stats = {
        'layer': conv_layer,
        'mean': np.mean(mi_scores),
        'std': np.std(mi_scores),
        'max': np.max(mi_scores),
        'min': np.min(mi_scores),
        'median': np.median(mi_scores),
        'values': mi_scores
    }
    print(summary_stats)
    all_stats.append(summary_stats)

df_stats = pd.DataFrame(all_stats)
#df_stats.to_csv('mis_stats_'+str(OPTION)+'.csv', index=False)

10/10 [==============================] - 9s 7ms/step
{'layer': 'block1_conv1', 'mean': 0.012323556780551859, 'std': 0.03349818851806473, 'max': 0.1601683558959106, 'min': 0.0, 'median': 0.0, 'values': array([0.        , 0.07961347, 0.03820432, 0.        , 0.        ,
       0.        , 0.        , 0.16016836, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.04166924, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.07469843,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])}
10/10 [==============================] - 0s 14ms/step
{'layer': 'block1_conv2', 'mean': 0.018310068891108644, 'std': 0.041185477989734515, 'max': 0.15854051886366127, 'min': 0.0, 'median': 0.0, 'values': array([0.00415934, 0.03955856, 0.        , 0.01402009, 0.        ,
       0.13209335, 0.08574904, 0.        , 0.15854052, 0.        ,
       0.        , 0.     

In [10]:
def renombrar_layer(layer):
    # Block: blockX_*
    if 'block' in layer:
        index = int(layer.split('_')[0][5:])
        return f'Block {str(index).zfill(2)}'
    # Conv: conv2d o conv2d_X
    if 'conv2d' in layer:
        index = layer[7:]
        index = 0 if index == "" else index
        return f'Conv {int(index)+1}'
    # Dense (si existiera)
    if 'dense' in layer:
        return 'Dense'
    # Si no encaja en los patrones anteriores, dejar el nombre original
    return layer

df_stats['layer_group'] = df_stats['layer'].apply(renombrar_layer)

In [11]:
def concat_list(x):
    # Elimina corchetes y saltos de línea
    #val_clean = x.values[0].replace('[', '').replace(']', '').replace('\n', ' ')
    # Convierte a array de floats
    print(x)
    arr = np.fromstring(x, sep=' ')
    return arr

valores_por_grupo = df_stats.groupby('layer_group')['values'].apply(lambda listas: np.concatenate(listas.values))

In [12]:
# Calcula las estadísticas por grupo
estadisticas = valores_por_grupo.apply(
    lambda x: pd.Series({
        'mean': np.mean(x),
        'std': np.std(x),
        'max': np.max(x)
    })
)

estadisticas.to_csv('mis_stats_'+str(OPTION)+'.csv')

In [13]:
estadisticas

,mean,std,max
layer_group,,,
Block 01,0.016315,0.038895,0.160168
Block 02,0.021036,0.047321,0.437857
Block 03,0.056165,0.064780,0.422097
Block 04,0.042041,0.044258,0.163746
Block 05,0.020898,0.036815,0.322797
Block 06,0.106408,0.043462,0.256863
Block 07,0.086311,0.050378,0.241769
Block 08,0.054780,0.059148,0.254418
Block 09,0.013814,0.034899,0.251766
